# 01. extract aa and nucleotide sequences to get ready for orthogrouping

In [ ]:
conda activate dl20-orthogrouping

In [ ]:
dl20=(
at6137 at6923 at6929 at7143
at8285 at9104 at9336 at9503
at9578 at9744 at9762 at9806
at9830 at9847 at9852 at9879
at9883 at9900
)

In [ ]:
dl20=(
at6923 at6929 at7143
at8285 at9104 at9336 at9503
at9578 at9744 at9762 at9806
at9830 at9847 at9852 at9879
at9883 at9900
)

In [ ]:
dl20=(
at9744 at9762 at9806
at9830 at9847 at9852 at9879
at9883 at9900
)

First process the scaffolds such that agat can cope

In [ ]:
mkdir -p input/line_wrapped_scaffolds

for accession in "${dl20[@]}"
do 
fasta_formatter -i ../assembly-and-annotation/output/01_assembly/03_inversion_fixed/${accession}.fasta \
   -w 60 \
   > input/line_wrapped_scaffolds/${accession}.fasta
done

## First extract the amino acid sequences and nucleotide sequences for all genes (not pseudogenes)

In [ ]:
mkdir -p output/per_accession_aa_cds

for accession in "${dl20[@]}"
do 
agat_sp_extract_sequences.pl \
   -g ../final/annotation/latest/${accession}_hodgepodgemerged.representatives.gff3 \
   -f input/line_wrapped_scaffolds/${accession}.fasta \
   -t cds \
   -p \
   -o output/per_accession_aa_cds/${accession}_cds_aa.fasta
done

#check that this is just for the genes
#perhaps make a subset for this purpose with just the primary transcripts - kevin is already doing this

In [ ]:
mkdir -p output/per_accession_nt_cds

for accession in "${dl20[@]}"
do 
agat_sp_extract_sequences.pl \
   -g ../final/annotation/latest/${accession}_hodgepodgemerged.representatives.gff3 \
   -f input/line_wrapped_scaffolds/${accession}.fasta \
   -t cds \
   -o output/per_accession_nt_cds/${accession}_cds_nt.fasta
done

## also extract the nucleotide sequences for the gene regions with introns (and utr if it's there)

In [ ]:
mkdir -p output/per_accession_nt_gene

for accession in "${dl20[@]}"
do 
agat_sp_extract_sequences.pl \
   -g ../final/annotation/latest/${accession}_hodgepodgemerged.gff3 \
   -f input/line_wrapped_scaffolds/${accession}.fasta \
   -t gene \
   -o output/per_accession_nt_gene/${accession}_gene_nt.fasta
done

In [ ]:
#agat_sp_extract_sequences.pl -g infile.gff -f infile.fasta -t exon --merge

mkdir -p output/per_accession_nt_mRNA

for accession in "${dl20[@]}"
do 
agat_sp_extract_sequences.pl \
   -g ../final/annotation/latest/${accession}_hodgepodgemerged.gff3 \
   -f input/line_wrapped_scaffolds/${accession}.fasta \
   -t exon --merge\
   -o output/per_accession_nt_mRNA/${accession}_mRNA_nt.fasta
done

## and extract the nucleotide sequences for the pseudogenes (for whole region)

In [ ]:
#agat_sp_extract_sequences.pl -g infile.gff -f infile.fasta -t pseduogene
#agat_sp_extract_sequences.pl -g infile.gff -f infile.fasta -t pseduogenic_region

mkdir -p output/per_accession_nt_pseudogene

for accession in "${dl20[@]}"
do 
agat_sp_extract_sequences.pl \
   -g ../final/annotation/latest/${accession}_hodgepodgemerged.gff3 \
   -f input/line_wrapped_scaffolds/${accession}.fasta \
   -t pseudogene \
   -o output/per_accession_nt_pseudogene/${accession}_pseudogene_nt.fasta
done

In [ ]:
mkdir -p output/per_accession_nt_pseudogenic_region

for accession in "${dl20[@]}"
do 
agat_sp_extract_sequences.pl \
   -g ../final/annotation/latest/${accession}_hodgepodgemerged.gff3 \
   -f input/line_wrapped_scaffolds/${accession}.fasta \
   -t pseudogenic_region \
   -o output/per_accession_nt_pseudogenic_region/${accession}_pseudogenic_region_nt.fasta
done

# Here down are some of the steps performed to validate the manual annotations

### Code to check the mannual annotation table and check if there is transdecoder present

In [ ]:
mkdir -p tmp/per_accession_nlr_transdecoder_overlap

for accession in "${dl20[@]}"
do
bedtools intersect \
   -a ../gffuuuu/output/${accession}_nlr_decisions.bed \
   -b ../gffuuuu/input/${accession}.tama-v2.fix.gff \
   -u \
   > tmp/per_accession_nlr_transdecoder_overlap/${accession}_nlrs_with_transcripts.txt
done

In [ ]:
cat tmp/per_accession_nlr_transdecoder_overlap/*_nlrs_with_transcripts.txt > tmp/per_accession_nlr_transdecoder_overlap/all_nlrs_with_transcripts.txt

In [ ]:
for accession in "${dl20[@]}"
do
grep \
   "aug_liftoff_cds_disagree_use_aug" ../gffuuuu/output/${accession}_nlr_decisions.bed | wc -l
done

In [ ]:
mkdir -p tmp/per_accession_nlr_transdecoder_overlap

for accession in "${dl20[@]}"
do
bedtools intersect \
   -a ../gffuuuu/output/${accession}_nlr_decisions.bed \
   -b ../gffuuuu/input/${accession}.tama-v2.fix.gff \
   -u \
   > tmp/per_accession_nlr_transdecoder_overlap/${accession}_nlrs_with_transcripts.txt
done

### The code below was used to add the at9879 NLR annotator results that did not overlap with the already manually annotated loci

In [ ]:
bedtools intersect \
   -a ../NLRannotator/output/at9879_nlr_annotator.bed \
   -b ../gffuuuu/output/at9879_nlr_decisions.bed \
   -v \
   > tmp/at9879_nlr_annotator_extra.bed

In [ ]:
cat tmp/at9879_nlr_annotator_extra.bed |wc -l